# Prepare database

In [17]:
import os

dir_name = 'C:/Users/windo/Desktop/Inżynierka/plap/thesis/audios/nEMO'
# dir_name = 'C:/Users/windo/Documents/ravdess-dataset/audio_speech_actors_01-24'

# Initialize a list to hold the file names
all_files = []

# Traverse all subfolders and files using os.walk()
for root, _, file_names in os.walk(dir_name):
    for file_name in file_names:
        file_path = os.path.join(root, file_name)
        if os.path.isfile(file_path):  # Check if it's a file
            all_files.append(file_path)

all_files = sorted(all_files)

# # Display the collected file names
# print('List of all files in the folders:')
# for file in all_files:
#     print(file)

# Emotions in the RAVDESS dataset
emotions = {
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

# desired csv columns:
# label/name(actor number, take, etc) emotion(class) feature1 feature2...
# or class at the end?
len(all_files), all_files[50]

(4512,
 'C:/Users/windo/Desktop/Inżynierka/plap/thesis/audios/nEMO\\samples\\EB0_anger_27.wav')

# Downsample the database to 'neutral', 'happy' and 'sad'

In [ ]:
wanted_classes = {'01', '03', '04'}

filtered_files = [filepath for filepath in all_files if filepath.split('\\')[-1].split('-')[2] in wanted_classes]

len(filtered_files)

# Print the filtered filepaths
# for file in filtered_files:
#     print(file)
# VISUALIZATION
import matplotlib.pyplot as plt
import numpy as np
plt.figure(figsize=(35,4))
plt.subplot(1,3,1)
emotion_list, count = np.unique([filepath.split('\\')[-1].split('-')[2] for filepath in filtered_files], return_counts=True)
emotion_list, count
emotions_pl = {
  '01':'neutralność',
  '02':'spokój',
  '03':'szczęście',
  '04':'smutek',
  '05':'złość',
  '06':'strach',
  '07':'obrzydzenie',
  '08':'zaskoczenie'
}
plt.bar(x=range(3), height=count)
plt.xticks(ticks=range(3), labels = [emotions_pl[emotion] for emotion in emotion_list], fontsize=10)
plt.xlabel('Klasa')
plt.tick_params(labelsize=16)
plt.ylabel('Liczba próbek (obserwacji)')
# Add text annotations
for i, value in enumerate(count):
    plt.text(i, value + 1, str(value), ha='center', fontsize=10)
plt.show()

480

In [21]:
# filtered_files = [filepath for filepath in all_files if '.wav' in filepath and 'neutral' in filepath or 'happiness' in filepath or 'sadness' in filepath]
# # Downsample nEMO
# len(filtered_files)
# # wanted_classes = {'neutral', 'happiness', 'sadness'}
# # filtered_files = [filepath for filepath in all_files if filepath.split('\\')[-1].split('_')[1] in wanted_classes]

# # # Print the filtered filepaths
# # for file in filtered_files:
# #     print(file)
# # filtered_files = filtered_files[0:-1:5]
# len(filtered_files)

# VISUALIZATION
# import matplotlib.pyplot as plt
# import numpy as np
# plt.figure(figsize=(35,4))
# plt.subplot(1,3,1)
# temp = [filepath.split('\\')[-1].split('_')[1] for filepath in all_files if '.wav' in filepath]
# emotion_list, count = np.unique(temp, return_counts=True)
# emotion_list, count
# emotions_pl = {
#   'neutral':'neutralność',
#   'happiness':'szczęście',
#   'sadness':'smutek',
#   'anger':'złość',
#   'fear':'strach',
#   'surprised':'zaskoczenie'
# }
# #(anger, fear, happiness, sadness, surprised, and neutral)
# plt.bar(x=range(6), height=count)
# plt.xticks(ticks=range(6), labels = [emotions_pl[emotion] for emotion in emotion_list], fontsize=10)
# plt.xlabel('Klasa')
# plt.tick_params(labelsize=16)
# plt.ylabel('Liczba próbek (obserwacji)')
# # Add text annotations
# for i, value in enumerate(count):
#     plt.text(i, value + 1, str(value), ha='center', fontsize=10)
# plt.show()

In [ ]:
import numpy as np

import sys
sys.path.append("..")
import plap

# Use plap for parameterization
# Newest with ASB, ASP and variance across ASF
fvector = plap.FeatureVector("asc","asc_var","ass","ass_var","asf","asf_mean","asf_var","asf_var_mean","aff","aff_var","lat","tc","sc","sc_var","hsc","hsd","hss","hsv","asb","asb_mean","asp","asp_mean","mfcc")
# fvector = plap.FeatureVector("asc","asc_var","ass","ass_var","asf","asf_mean","aff","aff_var","lat","tc","sc","sc_var","hsc","hsd","hss","hsv")
# fvector = plap.FeatureVector("asc","ass","asf","lat","tc","sc","hsc","hsd","hss","hsv")
# block_size_ms = block_size_samples/sr*1000 = 1024/48000*1000 ~= 21.34 ms
preprocessor = plap.Preprocessor(preemphasis_coeff=None, block_size=1024, window_type="hamming")

def extract_features(audio_path):
    plap.parameterize(audio_path=audio_path, fvector=fvector, preprocessor=preprocessor)
    return fvector.values

In [ ]:
import csv

# Define the CSV filename
# fvector = plap.FeatureVector("asc","ass","asf","lat","tc","sc","hsc","hsd","hss","hsv")
csv_filename = './python_csv/nEMO_wholedataset.csv'

# Feature names compatibility taking into account vector features
feature_names_compat = [feature.upper() for feature in fvector.features]

# Define mapping for base features and their corresponding counts
feature_mapping = {
    'ASF': 24,
    'ASF_VAR': 24,
    'ASB': 20,
    'ASP': 21,
    'MFCC': 20
}

i = 0
while i < len(feature_names_compat):
    base_feature = feature_names_compat[i]
    
    if base_feature in feature_mapping:
        count = feature_mapping[base_feature]
        feature_names_compat[i] = f'{base_feature}1'  # Set the first feature name
        
        # Create the numbered features list
        numbered_features = [f'{base_feature}{k}' for k in range(2, count + 1)]
        
        # Insert numbered features right after the base feature
        feature_names_compat[i+1:i+1] = numbered_features
        
        # Skip over the newly inserted features
        i += len(numbered_features)
    
    i += 1

feature_names_compat
# Initialize the CSV file with headers
headers = ['label', 'class'] + [f'{feature.upper()}' for feature in feature_names_compat]
with open(csv_filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(headers)

In [ ]:
import time
# Iterate over each audio file
initial_i = 2327
# end_i = 150
i = initial_i
processed = 2327
for filename in filtered_files[initial_i:]:
    if filename.endswith('.wav'):  # Ensure only audio files are processed
        
        start_time = time.time()
        # Extract label and class from the filename (customize this logic as needed)
        # audio_filename = filename.split('\\')[-1]
        # label = audio_filename.removesuffix('.wav')  # Use filename as label
        # class_num = audio_filename.split('-')[2]
        # class_name = emotions[class_num]
        audio_filename = filename.split('\\')[-1]
        label = audio_filename.removesuffix('.wav')
        class_name = audio_filename.split('_')[1]
        
        print(f'Processing file {audio_filename}')

        # Extract 11 features from the audio file
        try:
            features = extract_features(filename)
            processed += 1
        except:
            print(f'Omitting file{i}')
            i += 1
            continue
        # features = np.zeros(11)

        # Prepare the row to be added to the CSV
        row = [label, class_name] + list(features)

        # Append the new row to the CSV file
        with open(csv_filename, mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(row)
        print(f'Finished file{i} in {time.time() - start_time} s')
        i += 1

print("Feature extraction and CSV writing completed.")

In [ ]:
processed